In [32]:
#!pip uninstall --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#!pip install pydrive
!pip3 install google-api-python-client google-auth-httplib2 google-auth-oauthlib tabulate requests tqdm  Copy


You should consider upgrading via the '/Users/burkeatilla/Source/venv/bin/python3 -m pip install --upgrade pip' command.


NameError: name 'Copy' is not defined

In [ ]:
#some of the code is based on https://www.thepythoncode.com/article/using-google-drive--api-in-python
# to create a application for yourself follow https://developers.google.com/drive/api/quickstart/python60/2.5 

In [34]:
from tabulate import tabulate


In [18]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly',          
          'https://www.googleapis.com/auth/drive.file']


In [27]:
"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
    else:    
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')

Files:
03.04.2010 - 30.04.2010 [GELIBOLU] (1JwiiXWknLSbjfR91u2syRFFbbO6n3oBp)
23.05.2010 - 18.06.2010 (1wr5Diht94wan-bGkwvy-x-qBLBi0SOc1)
17.05.2010 - 22.05.2010[KAAN DOGUM] (1yLH_MjF9C4W_N3SyaaenAGoyPKho_iE-)
HillSide BEACH CLUB 2010 (1-l4EgOZp6MUOztzeRkJX7j6HILO4M_Q4)
BA (1Aaz7NpVBK4yxWeaIWU0nh9-iebmQc_sF_RDxDxBNYWM)
Reference-Architecture-Mapping.drawio.png (1DZyQ2ns1IA_5JKP0B_djeW_Wo7IXaGKS)
~$Reference-Architecture-Mapping.drawio.png.dtmp (1DkXyKMSaTK_Ldr6O7zdcO2lkiLofJaWP)
~$Reference-Architecture-Mapping.drawio.png.dtmp (1DkVPnFUtOoLccDqJXw6R2mmkqgvt08tg)
101P_001 (1FqTj52LW02XyQ7NwL6dExF9-9JeZ4mGC)
102NIKON (1E4hu_nVebHz5xzjHuTYaZhcxI2ZP2mRZ)


In [29]:
DRIVE = discovery.build('drive', 'v3', credentials=creds)

files = DRIVE.files().list().execute().get('files', [])
for f in files:
    print(f['name'], f['mimeType'])

03.04.2010 - 30.04.2010 [GELIBOLU] application/vnd.google-apps.folder
23.05.2010 - 18.06.2010 application/vnd.google-apps.folder
17.05.2010 - 22.05.2010[KAAN DOGUM] application/vnd.google-apps.folder
HillSide BEACH CLUB 2010 application/vnd.google-apps.folder
BA application/vnd.google-apps.spreadsheet
Reference-Architecture-Mapping.drawio.png image/png
~$Reference-Architecture-Mapping.drawio.png.dtmp application/octet-stream
~$Reference-Architecture-Mapping.drawio.png.dtmp application/octet-stream
101P_001 application/vnd.google-apps.folder
102NIKON application/vnd.google-apps.folder
100NIKON application/vnd.google-apps.folder
15.03.2010 [Doğumgünü] application/vnd.google-apps.folder
Dubai2010 application/vnd.google-apps.folder
28.03.2010 application/vnd.google-apps.folder
20.03.2010 -27.03.2009 application/vnd.google-apps.folder
DrawIo application/vnd.google-apps.folder
Centralized-Logging-2.png image/png
Reference-Architecture-RA-0000.drawio.png image/png
Reference-Architecture.drawi

In [30]:
def list_files(items):
    """given items returned by Google Drive API, prints them in a tabular way"""
    if not items:
        # empty drive
        print('No files found.')
    else:
        rows = []
        for item in items:
            # get the File ID
            id = item["id"]
            # get the name of file
            name = item["name"]
            try:
                # parent directory ID
                parents = item["parents"]
            except:
                # has no parrents
                parents = "N/A"
            try:
                # get the size in nice bytes format (KB, MB, etc.)
                size = get_size_format(int(item["size"]))
            except:
                # not a file, may be a folder
                size = "N/A"
            # get the Google Drive type of file
            mime_type = item["mimeType"]
            # get last modified date time
            modified_time = item["modifiedTime"]
            # append everything to the list
            rows.append((id, name, parents, size, mime_type, modified_time))
        print("Files:")
        # convert to a human readable table
        table = tabulate(rows, headers=["ID", "Name", "Parents", "Size", "Type", "Modified Time"])
        # print the table
        print(table)

In [46]:
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"

In [35]:
    # Call the Drive v3 API
    results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
    items = results.get('files', [])
    # list all 20 files & folders
    list_files(items)

Files:
ID                                            Name                                 Parents                                Size    Type                                     Modified Time
--------------------------------------------  -----------------------------------  -------------------------------------  ------  ---------------------------------------  ------------------------
1JwiiXWknLSbjfR91u2syRFFbbO6n3oBp             03.04.2010 - 30.04.2010 [GELIBOLU]   ['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']  N/A     application/vnd.google-apps.folder       2022-03-16T19:08:55.885Z
1wr5Diht94wan-bGkwvy-x-qBLBi0SOc1             23.05.2010 - 18.06.2010              ['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']  N/A     application/vnd.google-apps.folder       2022-03-16T19:08:55.885Z
1yLH_MjF9C4W_N3SyaaenAGoyPKho_iE-             17.05.2010 - 22.05.2010[KAAN DOGUM]  ['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']  N/A     application/vnd.google-apps.folder       2022-03-16T19:08:55.885Z
1-l4EgOZp6MUOztzeRkJX7j6

In [ ]:
def upload_files():
    """
    Creates a folder and upload a file to it
    """
    # authenticate account
    service = get_gdrive_service()
    # folder details we want to make
    folder_metadata = {
        "name": "TestFolder",
        "mimeType": "application/vnd.google-apps.folder"
    }
    # create the folder
    file = service.files().create(body=folder_metadata, fields="id").execute()
    # get the folder id
    folder_id = file.get("id")
    print("Folder ID:", folder_id)
    # upload a file text file
    # first, define file metadata, such as the name and the parent folder ID
    file_metadata = {
        "name": "test.txt",
        "parents": [folder_id]
    }
    # upload
    media = MediaFileUpload("test.txt", resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print("File created, id:", file.get("id"))

In [ ]:
def search(service, query):
    # search for the file
    result = []
    page_token = None
    while True:
        response = service.files().list(q=query,
                                        spaces="drive",
                                        fields="nextPageToken, files(id, name, mimeType)",
                                        pageToken=page_token).execute()
        # iterate over filtered files
        for file in response.get("files", []):
            result.append((file["id"], file["name"], file["mimeType"]))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            # no more files
            break
    return result


In [ ]:
def download():
    service = get_gdrive_service()
    # the name of the file you want to download from Google Drive 
    filename = "bbc.zip"
    # search for the file by name
    search_result = search(service, query=f"name='{filename}'")
    # get the GDrive ID of the file
    file_id = search_result[0][0]
    # make it shareable
    service.permissions().create(body={"role": "reader", "type": "anyone"}, fileId=file_id).execute()
    # download file
    download_file_from_google_drive(file_id, filename)

In [ ]:
def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value
        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768
        # get the file size from Content-length response header
        file_size = int(response.headers.get("Content-Length", 0))
        # extract Content disposition from response headers
        content_disposition = response.headers.get("content-disposition")
        # parse filename
        filename = re.findall("filename=\"(.+)\"", content_disposition)[0]
        print("[+] File size:", file_size)
        print("[+] File name:", filename)
        progress = tqdm(response.iter_content(CHUNK_SIZE), f"Downloading {filename}", total=file_size, unit="Byte", unit_scale=True, unit_divisor=1024)
        with open(destination, "wb") as f:
            for chunk in progress:
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # update the progress bar
                    progress.update(len(chunk))
        progress.close()

    # base URL for download
    URL = "https://docs.google.com/uc?export=download"
    # init a HTTP session
    session = requests.Session()
    # make a request
    response = session.get(URL, params = {'id': id}, stream=True)
    print("[+] Downloading", response.url)
    # get confirmation token
    token = get_confirm_token(response)
    if token:
        params = {'id': id, 'confirm':token}
        response = session.get(URL, params=params, stream=True)
    # download to disk
    save_response_content(response, destination)  

In [45]:
mimeType = 'application/vnd.google-apps.folder'
page_token = None
while True:
    response = service.files().list(q="mimeType = 'application/vnd.google-apps.folder'",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name,parents)',
                                          pageToken=page_token).execute()
    for file in response.get('files', []):
        # Process change
        print ('Found file: %s (%s)-[%s]' % (file.get('name'), file.get('id'),file.get('parents')))
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break

Found file: 03.04.2010 - 30.04.2010 [GELIBOLU] (1JwiiXWknLSbjfR91u2syRFFbbO6n3oBp)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: 23.05.2010 - 18.06.2010 (1wr5Diht94wan-bGkwvy-x-qBLBi0SOc1)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: 17.05.2010 - 22.05.2010[KAAN DOGUM] (1yLH_MjF9C4W_N3SyaaenAGoyPKho_iE-)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: HillSide BEACH CLUB 2010 (1-l4EgOZp6MUOztzeRkJX7j6HILO4M_Q4)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: 101P_001 (1FqTj52LW02XyQ7NwL6dExF9-9JeZ4mGC)-[['1cWwy6SO4zWgvDL-TX0hayxdubpAq68z1']]
Found file: 102NIKON (1E4hu_nVebHz5xzjHuTYaZhcxI2ZP2mRZ)-[['1cWwy6SO4zWgvDL-TX0hayxdubpAq68z1']]
Found file: 100NIKON (1z9duMDqoAiy9qDon7Z5F33mLIckiM4km)-[['1cWwy6SO4zWgvDL-TX0hayxdubpAq68z1']]
Found file: 15.03.2010 [Doğumgünü] (1I_RwzL6hA3ERm5PMKQ6RI13okEph8yAV)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: Dubai2010 (1cWwy6SO4zWgvDL-TX0hayxdubpAq68z1)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: 28.03.2010 (1osf

Found file: 15.06-23.06.2007 (1AAFZckLd9xBeRY6mD_QYaQ_J-71_p-OP)-[['1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU']]
Found file: .AppleDouble (1Zo0NM1ggEAqsZBx2XItbNW8TzVFFaKES)-[['1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU']]
Found file: 11.04-15.04.2007 (1ctzUoE-REfF7umDdHNWtzzgxTslKz0On)-[['1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU']]
Found file: Unclassified (1wzXgNIhcqJTYCKDFNDzoL-aOg3Jv5rTX)-[['1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU']]
Found file: 25.04-03-05.2007 (1_RT0qf9rtn0LxOPf1UzEf14-WbpR_5Su)-[['1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU']]
Found file: 13.05-29.05.2007 (1RYMoIai8ZfBoWCpiEYuOrBtzOII5riJZ)-[['1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU']]
Found file: BADE_DVD1 (1xz1JXsG-3uNGCG_h9GArgAP7Icf2K1XU)-[['1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG']]
Found file: Pictures (1oTGPQ1CDaRRdGdMqPFjxJZrc4ZLfcVbG)-[['0ALgHLBxuoLbqUk9PVA']]
Found file: 05.06.2012 (1sVyluwOGESr4oJ0dQFng4T8-7pA8z80-)-[['1yLXCpsQy7V83TOtVtu_igTzE0VV2B4IQ']]
Found file: SoftTech (1nqQILtrqY2ltOEeGr_msRIzOlzhJEXLi)-[['1yLXCpsQy7V83TOtVtu_igTzE0VV2B4IQ']]
F

Found file: __pycache__ (1rmn0a4DXSYceeC1KRYrMSkGJt_6UUH66)-[['1ezeTnKQ-OTDScEJwwN445LWgLO1X-iip']]
Found file: indexing (1ezeTnKQ-OTDScEJwwN445LWgLO1X-iip)-[['1IfLymnfJ4tbCpTQPjjMtG8afS3bfZBnR']]
Found file: __pycache__ (1rsqzoHqyKxTD0mgPvJs3V0IRxH2huudX)-[['1eMg0aRAAWYsd9jKGC5WgIfxvzVRUNQBY']]
Found file: accessors (1eMg0aRAAWYsd9jKGC5WgIfxvzVRUNQBY)-[['1IfLymnfJ4tbCpTQPjjMtG8afS3bfZBnR']]
Found file: series (1IfLymnfJ4tbCpTQPjjMtG8afS3bfZBnR)-[['1qp4ZUujbEQ5t644yJdwOMawGz7BJoXSP']]
Found file: __pycache__ (1bxYq91KjSTQi8wGXAUzuv6eS-9eG_u4v)-[['1WYwib6vZm_HbyM_lUMLmYU1fK_hMlO1U']]
Found file: __pycache__ (1bG-DKUouGdi9dcbXh6WUxSn1Z0WxEQdk)-[['1WbP0KvfeWaaW_dRKMPTvUmM7_D-zSkj5']]
Found file: timestamp (1WYwib6vZm_HbyM_lUMLmYU1fK_hMlO1U)-[['1IJZQK-NH29YDl1pQ0AiouFCwcaI1ALiW']]
Found file: timedelta (1WbP0KvfeWaaW_dRKMPTvUmM7_D-zSkj5)-[['1IJZQK-NH29YDl1pQ0AiouFCwcaI1ALiW']]
Found file: __pycache__ (1WZmlCk-8rjCqGH43aq-35tcGG4XWDjem)-[['1IJZQK-NH29YDl1pQ0AiouFCwcaI1ALiW']]
Found file: __

Found file: generic (1IKy6CS1fHwkReagm3QhkN3GWMK8028ca)-[['1qp4ZUujbEQ5t644yJdwOMawGz7BJoXSP']]
Found file: __pycache__ (1vRjAOlKcsJ-eA_alEt-zLgNEDgo7PyP1)-[['1J7m38fOUECXfVOgyJyo_Ao4lkpa58n5o']]
Found file: __pycache__ (12mvOdap2xEWedjXBiVoADJtecBkmftPg)-[['1vNVyEzg154kQt9f0Fq-SHiCtvkVuisoU']]
Found file: methods (1vNVyEzg154kQt9f0Fq-SHiCtvkVuisoU)-[['1J7m38fOUECXfVOgyJyo_Ao4lkpa58n5o']]
Found file: __pycache__ (1-m5b2eP_cjdCfbB7o9of1Ua1bnsW-pYZ)-[['1vWGbq5RRnC8oqfWiUh9v4nIXC2wpQ4-e']]
Found file: indexing (1vWGbq5RRnC8oqfWiUh9v4nIXC2wpQ4-e)-[['1J7m38fOUECXfVOgyJyo_Ao4lkpa58n5o']]
Found file: __pycache__ (10DB5gX0UVSTd1YqjUGggUdtkcRpSYHue)-[['1vTMj0f3gz9PFCyIJcfOtkjmtaAC-ZAMd']]
Found file: constructors (1vTMj0f3gz9PFCyIJcfOtkjmtaAC-ZAMd)-[['1J7m38fOUECXfVOgyJyo_Ao4lkpa58n5o']]
Found file: __pycache__ (1TEPkMOoHWbezYjV1GAXN8Zt60L3RyZUc)-[['1I6oiqUSABHHpoJtiEW-VthJadPtSF9Om']]
Found file: frame (1J7m38fOUECXfVOgyJyo_Ao4lkpa58n5o)-[['1qp4ZUujbEQ5t644yJdwOMawGz7BJoXSP']]
Found file: __py

Found file: __pycache__ (1PMhEEmD8oJFiQIrzvvwy9vzfnNE2wGHa)-[['1I4SXdnNc90vxJzSqeEwPs_24w_41ILd9']]
Found file: internals (1I4SXdnNc90vxJzSqeEwPs_24w_41ILd9)-[['1qtJuDrGJEGSSx4ajhWt6bxd7XqXRQLzE']]
Found file: __pycache__ (1Kms-WdgNW3WRd7SyyKzS5I2Dr5Os38A0)-[['1HjWfAojqMnWUcrBfV40h2BXafFKR4hCL']]
Found file: indexes (1HjWfAojqMnWUcrBfV40h2BXafFKR4hCL)-[['1qtJuDrGJEGSSx4ajhWt6bxd7XqXRQLzE']]
Found file: __pycache__ (1QQR0cui8ZUXq4CJP5jeYU5KHrw74hqG_)-[['1Hzbu54BXYACBMiG0NSflAfiOCr0V49aY']]
Found file: groupby (1Hzbu54BXYACBMiG0NSflAfiOCr0V49aY)-[['1qtJuDrGJEGSSx4ajhWt6bxd7XqXRQLzE']]
Found file: __pycache__ (1NrnwLPdAOo4jOAnq05Q8p5PsT_NB8QrL)-[['1HXdzps2SZLXme73lD2F5dUfiJKvFzUk-']]
Found file: dtypes (1HXdzps2SZLXme73lD2F5dUfiJKvFzUk-)-[['1qtJuDrGJEGSSx4ajhWt6bxd7XqXRQLzE']]
Found file: __pycache__ (1PDu5fbC_4kqFfa3qJNWEPqJd7CxWcuhT)-[['1HidqyeA7UYe4jSm_ibWIKZIrPQzpvpE0']]
Found file: computation (1HidqyeA7UYe4jSm_ibWIKZIrPQzpvpE0)-[['1qtJuDrGJEGSSx4ajhWt6bxd7XqXRQLzE']]
Found file: __p

...

Found file: MetisSupport (1NxUk4ulNc_kngmQEz6v91Q32UbcsbiCv)-[['1L96-01QYVS3j62oWvNn_tW5cHFNqFmF4']]
Found file: misc (1MocKpTf82VF-Ta_M_AwtP0o0D9LXyTEY)-[['1L96-01QYVS3j62oWvNn_tW5cHFNqFmF4']]
Found file: task1 (16tTpBdNcqDQcRaJUjH44uD66eGa_O-WW)-[['14TBIaNWsbGeOGl9agq575x7bLRyjRFfs']]
Found file: project.xcworkspace (14HFniF6FRFN86b6_4igxqGp8wQTwoNJ1)-[['11w1aDhOiXXc70G1Z6jB7603A5ToqHR_X']]
Found file: matrix (1891SbVtJq9elVdSfkgLA8jY3bVbBYX0g)-[['16tkBArrzLUDuW7rzqEbzqiXN57VGvg3k']]
Found file: CUDA (1VET_YNGA3kVd7kBQ-QhNY3ZUJ79GiMTt)-[['1TMv3ccOxJNFv4NKbIzGupR9_tRJPjFfy']]
Found file: Math (1rP_qXtrVc3ytu0n6Fs8jtgpMFw6uO5Ne)-[['1r-nFD3B-c-1Za_sEhCPBOrqXaEGsjK9z']]
Found file: Drawing (1rRW_no4ISBaDMH2umyA53U6ixSVwqBiK)-[['1r-nFD3B-c-1Za_sEhCPBOrqXaEGsjK9z']]
Found file: NEON (1VNAahRBpyXWKASuhbYCVl7bj_bvp8h20)-[['1TMv3ccOxJNFv4NKbIzGupR9_tRJPjFfy']]
Found file: functors (1YilniZUOpuEayEoiCk05xICdxEEdr9_x)-[['1NP_l3ipI_zyGRqczOemexfbqianDsI6E']]
Found file: .vscode (1522CKyEL564GnFu

Found file: c7 (1hdpqNtZviMaeu3BAIafa7XE5IPs4pc1B)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 16 (1hNXacw0qZlR5vbbws80zemKcFQybqZTL)-[['1cihYrnjaRZbL-aNcyfCIo8O68_7ZpYS6']]
Found file: d7 (1geJ2fkqO4zCRHZBaiPbd7IwPyxSZuYV8)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 3f (1iuZjCW52-me4ZG3RsX5y3hOf37tS6oFV)-[['1cihYrnjaRZbL-aNcyfCIo8O68_7ZpYS6']]
Found file: f4 (1fZoro5LEh22eVf_bHpx_MtjcPc6EVSH4)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 3f (1gwy4IBZoxhyPhc9OdcoUo8WhGqsWDCDw)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 16 (1gNFmIs_F1MLMbdjty88naasASngevHmU)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: b0 (1gLdzGbf0gH5A6X-b4u1lozWNC4hOjxlf)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 65 (1hXfUgrqnZ2N0UOdzdXZm9T93AvecI59y)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 5b (1fjnz6sR4YXBk-5wu-iP5k8FA_bm6R0Z6)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]
Found file: 0f (1hcqA3KvFh8JsnQ9DUQ52JS7y4A5KfjBF)-[['1clC08oXRL3BjiDWix0GHvoV3ywmmT8HU']]

Found file: level3 (1iCVJa36eYpXPOUDgYHodHvbVToY7ZOOj)-[['1PNOY0wjicq0iS-hLawZCb7GaJY6XPLSF']]
Found file: c5 (1m2Zqg4eN5iSr86RAGYzZdRQ9qmJFYOGW)-[['1iCVJa36eYpXPOUDgYHodHvbVToY7ZOOj']]
Found file: Resources (1eDT1ygOqnDB_cbzfTlLsQjBGo77f2EDe)-[['1dxIKofAyNemHI5bMtXE0PRHIPzhtyaH8']]
Found file: 19 (1kAh44ctM2n_LL1XqFVAPlq-MIz-EorbZ)-[['1iQm34B4kB7wP6_dvTmpwW8lKs5YErVKm']]
Found file: 0f (1jfGXpYHtTXFJ_MI5AZkhEWhoT-q3y-Jx)-[['1iQm34B4kB7wP6_dvTmpwW8lKs5YErVKm']]
Found file: 16 (1j9wFWiHjyY9EMne_KBVt6aMC-bnDXONv)-[['1iQm34B4kB7wP6_dvTmpwW8lKs5YErVKm']]
Found file: f4 (1JMmOG8fE26mEADhuzsQZEBcQLPpTUd9c)-[['1I_bFvJwb3panmzVqLSPMmX2OfyXkIqkP']]
Found file: WrldResources (1JNrr-KiL7pRf_OzF7bQFllDluRNSelww)-[['1R6SuVWATXyKtF5j7cqt4sikNTsIqWDrD']]
Found file: Browsers (1EseklGfiOkVFfjJAEjpd3P-D4wJoAW1q)-[['1A5IqcAV3bxAM3ms8Zj06wLSLCQyoyNeW']]
Found file: Resources (1KKGapCis3FfgZioHRicBLXVumSxvQNzV)-[['1JNrr-KiL7pRf_OzF7bQFllDluRNSelww']]
Found file: GI (1Qtu8KBxVr3k1fxOHm6r3IbiL-vn35L9C)-[['1

Found file: dc (1tWGikjYauDFJxX-W5qQBht4_r7Ii3yaN)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: 3f (1lpGWpc1JWhjqCedrga37FXJDO3lyFoIA)-[['1VOawWIayJwGaM01J9fgikUCJpUoJbqzD']]
Found file: Browsers (1J2Tnmm-_28Djj_BuRcd6-2upPGY39XFl)-[['1ITR0Cp_3Po_ua0gyXi0cH7M0GpCJzWR3']]
Found file: EmbeddedTheme (1JvF2uQGd3ffpdr6vqn7hp_EvS8Pxd-PQ)-[['1IYzRdG9AF0NOzBi-hD3ke4RYdB8KFJKY']]
Found file: MacOS (1WUSKOQYzkFq_Y9FZ9YEegv44hHDEbfI2)-[['1VeJV54JAfCKBDhOV-WHxhMiCpTpGdpXX']]
Found file: MonoBleedingEdge (1H-2kPhPNLxs_WQnM45lcnuetz2Hu2Lfn)-[['1Ta6qcoK8IVqHE7SP35ym6NnhRsjGeikR']]
Found file: a9 (13vKREHboaULxxeucSZ272OxbOFSC_AiB)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: EmbeddedTheme (1ZNNEoK0zY6WE3WajvCalymqrhmcVosUa)-[['1WQYhr2Jc4KLUFL7RzoylP2lilnyWVDNS']]
Found file: e9 (1kleQZ8ZHEPsCky9Qd3TdO1ReHviWJew-)-[['1VOawWIayJwGaM01J9fgikUCJpUoJbqzD']]
Found file: Browsers (1W1wYa1ghrHvaykYq3MnHeIhN4NE7Hx7t)-[['1Va3ZaueCCeRlYC6PL-fz3apVVvoqJcDs']]
Found file: 9f (1Yh04KIBgs8cHhN4D41VkR3

Found file: etc (1GvM9KuCfYKYqp8WGO09F2-M5o3-axUb6)-[['1TYbUlGrSVHSA6HCZAMnxVXPg7p_FNdR7']]
Found file: f0 (12Gen13dcIhpmY6MyurtLNvNre5jJrgu2)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: ca (1ZEAaN3XUGl_e9ODXQ_Lu4slpUggSOuJJ)-[['1VOawWIayJwGaM01J9fgikUCJpUoJbqzD']]
Found file: f4 (1u6OCROAj4tT0bwl8Z_MxRE_ykXauFYgG)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: 9f (1tqpvCc4dOpJXs7lx4iMsaE0cbnHgJzMo)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: MacOS (1Td3HLWSiRkKWNorbqoEXMIo60ylviuvL)-[['1PLrDUnmwTplwq14g05GNntTMpGp5_tmW']]
Found file: 64 (1m2VpZgIJ8w5GpTKFQTapNTuNidtB_rtT)-[['1VOawWIayJwGaM01J9fgikUCJpUoJbqzD']]
Found file: f5 (13aFriEK1QOEbDfJmRmhdh9APdgTWeA3f)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: 4f (1mIkHW9lohsj3Tw-d5fTfUPO2Shu06R50)-[['1VOawWIayJwGaM01J9fgikUCJpUoJbqzD']]
Found file: c7 (13duI2LNcNAn4oTWSTwqpC7i2TDXo8TF5)-[['1VA8D2EgoeOFtfamqEH6lYonIkW1gp8fs']]
Found file: FCND-Simulator-Releases-0.1.1-sim (1uLNZOYAhA5Vv6bkMjg7hgaXIhS6_Wuav)-[['1